# Typeclasses

## IGNORE THIS (config and deps)

In [1]:
:l ../original-source/Course/Core.hs
:l ../original-source/Course/Optional.hs
:l ../original-source/Course/List.hs
import qualified Prelude as P
import Prelude (String)
import Data.Foldable (for_, traverse_)
import Test.QuickCheck

runTests :: [(P.String, Property)] -> IO ()
runTests tests = for_ tests $ \(name, prop) -> do
  P.putStrLn $ "Testing: " P.++ name
  quickCheck prop

threeOf :: Gen a -> Gen (a, a, a)
threeOf g = do
  x <- g
  y <- g
  z <- g
  P.return (x, y, z)

listOf :: Gen a -> Gen (List a)
listOf g =
  let
    genN 0 = P.return Nil
    genN n = do
      h <- g
      t <- genN (n - 1)
      P.return (h :. t)
  in do
    len <- choose (0, 100)
    genN len

## The Concept

- Typeclasses are *like* interfaces.
    - In Java and other such OO languages, anothing thing, also called "classes" are used to
      provide interfaces. While they share this *application*, classes from OOP are **completely different**
      from typeclasses.
- Typeclasses in FP are *primarily for* specifying [algebraic structures](https://en.wikipedia.org/wiki/Algebraic_structure) on types.
- Typeclasses in Haskell *are* a mechanism to *overload* a set of functions on multiple types.
    - In this regard they bare some similarity to interfaces
- This is done by specifying type signatures for functions that apply to types in the typeclass.

For more on typeclasses and how they differ from OOP classes, you can read [here](https://joyofhaskell.com/posts/2017-03-15-typeclasses-in-translation.html)

### Algebraic Structure?

There are exceptions, but by and large, typeclasses *tend* to be concerned with *combinations* of the following kinds of behaviour:

1. Given 2 or more values of this type, are there any functions I can use to produce another value in that type?
  - E.g `(+)`, `(*)`
2. Are there any particular values of this type that are special with respect to those functions?
  - E.g `0` is special for `(+)` and `1` is special for `(*)`.
3. Are there some "laws" that the functions obey that make reasoning about them easier?
  - Not enforced by the compiler, these laws don't exist as "citizens" in Haskell
    but are usually enforced with *Property Based Tests*.
  - E.g `(x + y) + z == x + (y + z) == x + y + z`


## Example

The following example is a simple typeclass, which doesn't really provide any "algebraic structure".

They are the exception to the rule, and are somewhat common in the wild.

In [2]:
class ToString a where
  toString :: a -> String

instance ToString Integer where
  toString = show

instance ToString Bool where
  toString True = "true"
  toString False = "false"

In [3]:
toString 3

"3"

In [4]:
toString True

"true"

In [5]:
:t toString

toString :: forall a. ToString a => a -> String

## Stop and Think/Discuss

- Keywords in typeclass declaration and instance declaration
- `ToString =>` in `:t toString` (can ignore `forall a.`)
- Anything else?

### Example: Monoid

Monoid is a typeclass that describes types whose values can be *aggregated*, and there is an "empty" value
that does not contribute to the aggregate.

#### Declaration

In [6]:
class Monoid a where
  mappend :: a -> a -> a
  mempty :: a

(<>) :: (Monoid a) => a -> a -> a
(<>) = mappend

infixr 6 <>

#### Laws

These can't be expressed in Haskell, so they dont exist anywhere in the Typeclass declaration, we usually
include them in documentation, and write property based tests to verify them.

In the future this will most likely be different, see a language like Idris for
examples of how this might look in the future with dependent types.

##### The law of Identity

`mempty` is both a *left* identity and a *right* identity to `mappend/<>`.

Left identity:

``` haskell
mempty <> x === x
```

Right identity:

``` haskell
x <> mempty === x
```

In [7]:
leftIdentityHolds :: (Monoid a, Show a, Eq a) => a -> Property
leftIdentityHolds x = mempty <> x === x

rightIdentityHolds :: (Monoid a, Show a, Eq a) => a -> Property
rightIdentityHolds x = x <> mempty === x

##### The law of Associativity

If we have 3 or more values, it doesn't matter what order we reduce the terms in:

``` haskell
(x <> y) <> z === x <> (y <> z) === x <> y <> z
```

This has a practical advantage in that if we have a large collection of values to
aggregate, we can split it up into subsequences and distribute the work. This is
a little analogous to map-reduce (prime candidates for map-reduce have a couple more
"properties" that make the book keeping simpler).

In [8]:
associativityLawHolds :: (Monoid a, Show a, Eq a) => a -> a -> a -> Property
associativityLawHolds x y z = ((x <> y) <> z) === (x <> (y <> z))

#### Example Instances

In [9]:
instance Monoid Integer where
  mempty = 0
  mappend = (+)
  
instance Monoid (List a) where
  mempty = Nil
  mappend = (++)

#### Verify the laws

In [10]:
verify :: IO ()
verify = runTests [
    ("integer left identity", forAll arbitrary leftIdentityHolds)
  , ("integer right identity", forAll arbitrary rightIdentityHolds)
  , ("integer associativity", forAll arbitrary associativityLawHolds)
  , ("list left identity", forAll (listOf arbitrary) leftIdentityHolds)
  , ("list right identity", forAll (listOf arbitrary) rightIdentityHolds)
  , ("list associativity", forAll (threeOf (listOf arbitrary)) (\(x, y, z) -> associativityLawHolds x y z))
  ]

verify

Testing: integer left identity
+++ OK, passed 100 tests.
Testing: integer right identity
+++ OK, passed 100 tests.
Testing: integer associativity
+++ OK, passed 100 tests.
Testing: list left identity
+++ OK, passed 100 tests.
Testing: list right identity
+++ OK, passed 100 tests.
Testing: list associativity
+++ OK, passed 100 tests.

## Final Thoughts

Typeclasses usually embody a concept into the *semantics* of the types into them.

In the case of Monoid, for example, the concept is *aggregation*, and the instance for a type describes how aggregation works for that particular type. Whenever I see `List` in a type, I know it can be aggregated,
and I know how it is aggregated, it becomes part of what `List` "is".

The idea of *identifying* algebraic structures that exist in the types we use is a one of the cornerstone fulcrums of the functional programming paradigm.

We generally workout how to create simple values of the type, and then combine them to form more complex values, and
the laws often afford us one or more of the following luxuries:

- Rewrite rules to simplify complex looking expressions in code
- Invariants that restrict the complexity of the behaviour
    - A law may imply that certain aspects of the behaviour never change when 2 or more values are combined.
    - Then we just have to verify that those aspects exist in the base "simple values" we start from and
    - We have to verify that the invariant is indeed preserved by the functions in the algebra.

For more on this aspect of the functional programming mindset, you can read [this blog post](http://www.haskellforall.com/2014/04/scalable-program-architectures.html)